In [ ]:
print('Setup complete.')

# JSON Schema & Retry Demo

## Learning Objectives
- Master structured output with JSON Schema
- Implement robust retry logic for API failures
- Handle malformed JSON responses gracefully
- Build production-ready error handling patterns

## The Challenge: Reliable Structured Output

Getting consistent, valid JSON from LLMs is crucial for building reliable applications. This demo shows how to enforce structure and recover from failures.

In [ ]:
# Install required packages
!pip install asksageclient pip_system_certs pydantic jsonschema rich tenacity tiktoken

In [ ]:
# ================================
# 🔐 Cell 1 — Load secrets (Colab) + pricing + token utils
# ================================
import os, time, csv
from typing import Optional, Dict
import tiktoken

from google.colab import userdata

ASKSAGE_API_KEY = userdata.get("ASKSAGE_API_KEY")
ASKSAGE_BASE_URL = userdata.get("ASKSAGE_BASE_URL")
ASKSAGE_EMAIL = userdata.get("ASKSAGE_EMAIL")

assert ASKSAGE_API_KEY, "ASKSAGE_API_KEY not provided."
assert ASKSAGE_EMAIL, "ASKSAGE_EMAIL not provided."

print("✓ Secrets loaded")
print("  • EMAIL:", ASKSAGE_EMAIL)
print("  • BASE URL:", ASKSAGE_BASE_URL or "(default)")

# Pricing (USD per 1,000,000 tokens)
PRICES_PER_M = {
    "gpt-5": {"input_per_m": 1.25, "output_per_m": 10.00},
    "gpt-5-mini": {"input_per_m": 0.25, "output_per_m": 2.00},
}

# Tokenizer
enc = tiktoken.get_encoding("o200k_base")

def count_tokens(text: str) -> int:
    return len(enc.encode(text or ""))

def cost_usd(model: str, input_tokens: int, output_tokens: int) -> float:
    if model not in PRICES_PER_M:
        raise ValueError(f"Unknown model: {model}")
    r = PRICES_PER_M[model]
    return (input_tokens / 1_000_000) * r["input_per_m"] + (output_tokens / 1_000_000) * r["output_per_m"]

In [ ]:
# ================================
# 🔧 Cell 2 — Import bootcamp_common and setup AskSage client
# ================================
import sys
sys.path.append('../../../')  # Adjust path to reach bootcamp_common

from bootcamp_common.ask_sage import AskSageClient

# Initialize AskSage client
client = AskSageClient(
    api_key=ASKSAGE_API_KEY,
    base_url=ASKSAGE_BASE_URL
)

print("✓ AskSage client initialized")

In [ ]:
import os
import json
import time
from typing import Dict, List, Optional, Any, Union
from dataclasses import dataclass
from datetime import datetime

import openai
import anthropic
from pydantic import BaseModel, ValidationError
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
from rich.console import Console
from rich.panel import Panel
from rich.table import Table
from rich.json import JSON

console = Console()
print("✅ Libraries loaded successfully")

## Define Structured Output Schema

We'll use Pydantic models to define our expected JSON structure.

In [ ]:
class SentimentAnalysis(BaseModel):
    """Structured sentiment analysis result"""
    text: str
    sentiment: str  # positive, negative, neutral
    confidence: float  # 0.0 to 1.0
    key_phrases: List[str]
    emotions: Dict[str, float]  # emotion -> intensity
    summary: str

class ProductReview(BaseModel):
    """Structured product review analysis"""
    overall_rating: int  # 1-5 stars
    aspects: Dict[str, int]  # aspect -> rating
    pros: List[str]
    cons: List[str]
    would_recommend: bool
    target_audience: str

@dataclass
class RetryStats:
    """Track retry statistics"""
    total_attempts: int = 0
    successful_attempts: int = 0
    json_parse_errors: int = 0
    validation_errors: int = 0
    api_errors: int = 0
    total_time: float = 0.0

# Test data
sample_reviews = [
    "This laptop is amazing! Super fast processor, beautiful display, and the battery lasts all day. Perfect for coding and gaming. Only downside is it gets a bit warm during heavy use.",
    "Terrible phone. Crashes constantly, camera quality is poor, and battery dies in 2 hours. Customer service was unhelpful. Would not recommend to anyone.",
    "Decent headphones for the price. Sound quality is good, comfortable to wear. Noise cancellation could be better. Good for casual listening but not for audiophiles."
]

print("📋 Schemas and test data ready!")

## Robust JSON Parser with Retry Logic

In [ ]:
class StructuredOutputClient:
    """Client for getting structured JSON output with retry logic"""
    
    def __init__(self):
        self.setup_clients()
        self.stats = RetryStats()
    
    def setup_clients(self):
        """Setup API clients with fallback to mock"""
        if os.getenv('OPENAI_API_KEY'):
            try:
                self.openai_client = openai.OpenAI()
                self.has_openai = True
                console.print("✅ OpenAI client configured")
            except Exception as e:
                self.has_openai = False
                console.print(f"⚠️ OpenAI setup failed: {e}")
        else:
            self.has_openai = False
            console.print("💡 No OpenAI API key, using mock responses")
    
    def extract_json_from_response(self, response_text: str) -> str:
        """Extract JSON from response that might have extra text"""
        # Try to find JSON block
        import re
        
        # Look for JSON in code blocks
        json_match = re.search(r'```(?:json)?\s*({.*?})\s*```', response_text, re.DOTALL)
        if json_match:
            return json_match.group(1)
        
        # Look for JSON object directly
        json_match = re.search(r'{.*}', response_text, re.DOTALL)
        if json_match:
            return json_match.group(0)
        
        # Return as-is if no extraction patterns match
        return response_text.strip()
    
    @retry(
        stop=stop_after_attempt(3),
        wait=wait_exponential(multiplier=1, min=1, max=10),
        retry=retry_if_exception_type((json.JSONDecodeError, ValidationError, openai.APIError))
    )
    def get_structured_output(self, prompt: str, schema: BaseModel, provider: str = "openai") -> Dict:
        """Get structured output with automatic retry on failures"""
        
        start_time = time.time()
        self.stats.total_attempts += 1
        
        try:
            # Generate schema description
            schema_json = schema.model_json_schema()
            schema_example = json.dumps(schema_json, indent=2)
            
            # Build structured prompt
            structured_prompt = f"""{prompt}

You must respond with valid JSON that follows this exact schema:

```json
{schema_example}
```

Important:
- Return ONLY valid JSON, no extra text
- Follow the schema exactly
- Use appropriate data types
- Include all required fields
"""
            
            # Get response from API or mock
            if self.has_openai and provider == "openai":
                response = self.openai_client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": structured_prompt}],
                    max_tokens=500,
                    temperature=0.3
                )
                response_text = response.choices[0].message.content
            else:
                # Mock response with occasional "failures" for demo
                import random
                if random.random() < 0.3 and self.stats.total_attempts == 1:  # 30% failure on first try
                    if random.choice([True, False]):
                        # Malformed JSON
                        response_text = "{'sentiment': 'positive', 'confidence': 0.8,}"  # Extra comma
                    else:
                        # Missing required field
                        response_text = '{"sentiment": "positive", "confidence": 0.8}'
                else:
                    # Good response
                    if "sentiment" in prompt.lower():
                        response_text = '''{
  "text": "Sample review text",
  "sentiment": "positive",
  "confidence": 0.85,
  "key_phrases": ["amazing", "fast processor", "beautiful display"],
  "emotions": {"joy": 0.8, "satisfaction": 0.9},
  "summary": "Highly positive review praising performance and design"
}'''
                    else:
                        response_text = '''{
  "overall_rating": 4,
  "aspects": {"performance": 5, "design": 4, "battery": 3},
  "pros": ["Fast", "Beautiful display", "Good for work"],
  "cons": ["Gets warm", "Price"],
  "would_recommend": true,
  "target_audience": "Professionals and gamers"
}'''
            
            # Extract and parse JSON
            json_text = self.extract_json_from_response(response_text)
            
            try:
                parsed_json = json.loads(json_text)
            except json.JSONDecodeError as e:
                self.stats.json_parse_errors += 1
                console.print(f"[red]JSON Parse Error:[/red] {e}")
                console.print(f"[yellow]Raw response:[/yellow] {json_text[:200]}...")
                raise
            
            # Validate against schema
            try:
                validated_data = schema(**parsed_json)
                self.stats.successful_attempts += 1
                self.stats.total_time += time.time() - start_time
                return validated_data.dict()
            except ValidationError as e:
                self.stats.validation_errors += 1
                console.print(f"[red]Validation Error:[/red] {e}")
                raise
        
        except Exception as e:
            if "API" in str(type(e)):
                self.stats.api_errors += 1
            self.stats.total_time += time.time() - start_time
            raise
    
    def get_stats_summary(self) -> Table:
        """Get retry statistics summary"""
        table = Table(title="Retry Statistics")
        table.add_column("Metric")
        table.add_column("Count")
        table.add_column("Rate")
        
        total = self.stats.total_attempts
        if total > 0:
            table.add_row("Total Attempts", str(total), "100%")
            table.add_row("Successful", str(self.stats.successful_attempts), 
                         f"{self.stats.successful_attempts/total*100:.1f}%")
            table.add_row("JSON Parse Errors", str(self.stats.json_parse_errors),
                         f"{self.stats.json_parse_errors/total*100:.1f}%")
            table.add_row("Validation Errors", str(self.stats.validation_errors),
                         f"{self.stats.validation_errors/total*100:.1f}%")
            table.add_row("API Errors", str(self.stats.api_errors),
                         f"{self.stats.api_errors/total*100:.1f}%")
            table.add_row("Total Time", f"{self.stats.total_time:.2f}s", 
                         f"{self.stats.total_time/total:.2f}s/attempt")
        
        return table

# Initialize client
client = StructuredOutputClient()
print("🔧 Structured output client ready!")

## Demo: Sentiment Analysis with Retry Logic

In [ ]:
# Test sentiment analysis with retry logic
console.print("\n🧪 [bold blue]Testing Sentiment Analysis with Retry Logic[/bold blue]\n")

for i, review in enumerate(sample_reviews[:2], 1):
    console.print(f"[yellow]Review {i}:[/yellow] {review[:100]}...")
    
    prompt = f"Analyze the sentiment of this product review: '{review}'"
    
    try:
        result = client.get_structured_output(prompt, SentimentAnalysis)
        
        console.print("[green]✅ Success![/green]")
        console.print(Panel(JSON.from_data(result), title="Structured Result", border_style="green"))
        
    except Exception as e:
        console.print(f"[red]❌ Final failure after retries: {e}[/red]")
    
    console.print("\n" + "-"*50 + "\n")

# Show retry statistics
console.print(client.get_stats_summary())

## Demo: Product Review Analysis

In [ ]:
# Reset stats for clean demo
client.stats = RetryStats()

console.print("\n🛍️ [bold blue]Testing Product Review Analysis[/bold blue]\n")

review = sample_reviews[0]  # The positive laptop review
prompt = f"""Analyze this product review and extract structured information:

Review: "{review}"

Extract:
- Overall rating (1-5 stars)
- Aspect ratings (performance, design, battery, etc.)
- Pros and cons lists
- Recommendation status
- Target audience"""

try:
    result = client.get_structured_output(prompt, ProductReview)
    
    console.print("[green]✅ Product analysis complete![/green]")
    console.print(Panel(JSON.from_data(result), title="Product Review Analysis", border_style="green"))
    
    # Show stats
    console.print("\n")
    console.print(client.get_stats_summary())
    
except Exception as e:
    console.print(f"[red]❌ Analysis failed: {e}[/red]")

print("\n🎯 Product review analysis complete!")

## Key Takeaways: Reliable Structured Output

### 🔧 **Production-Ready Patterns**

1. **Schema Definition**: Use Pydantic models for type safety and validation
2. **Retry Logic**: Implement exponential backoff for transient failures
3. **JSON Extraction**: Handle responses with extra text or formatting
4. **Error Classification**: Track different types of failures separately
5. **Monitoring**: Collect metrics on retry rates and success patterns

### 📊 **Common Failure Patterns**

| **Failure Type** | **Cause** | **Solution** |
|------------------|-----------|---------------|
| **Malformed JSON** | Extra commas, missing quotes | JSON extraction + parsing |
| **Schema Violations** | Missing fields, wrong types | Pydantic validation + retry |
| **API Errors** | Rate limits, timeouts | Exponential backoff |
| **Context Limits** | Response truncation | Shorter prompts, streaming |

### 🚀 **Best Practices**

- **Always validate** against schema before using data
- **Log failures** with context for debugging
- **Set reasonable retry limits** to avoid infinite loops
- **Use mock responses** for testing and development
- **Monitor success rates** in production

## Next: Plan-Do-Check Demo

Next we'll see how to build systematic workflows with validation loops!